# EM0122

## Imports

In [1]:
from ExperimentManagement.ExperimentHierarchy import ExperimentData
import numpy as np

## Generation

In [1]:
#

## Load Hierarchy

In [2]:
EH = ExperimentData.loadHierarchy("D:\\EM0122")

Loading Experimental Hierarchy...
Finished.
Activating auto-logging. Current session state plus future input saved.
Filename       : D:\EM0122\log_file.log
Mode           : append
Output logging : True
Raw input log  : True
Timestamping   : True
State          : active
Logging Initiated


## Image PreProcessing

Imports

In [3]:
from ImagingAnalysis.PreprocessingImages import PreProcessing

Define Directories

In [4]:
RawVideoDirectory = EH.Encoding.folder_dictionary.get("raw_imaging_data")
OutputDirectory = EH.Encoding.folder_dictionary.get("compiled_imaging_data_folder").path

Import MetaData

In [5]:
EH.Encoding.loadBrukerMetaData()

Repackage bruker files into a single tiff

In [5]:
PreProcessing.repackageBrukerTiffs(RawVideoDirectory, OutputDirectory)
EH.Encoding.folder_dictionary.get("compiled_imaging_data_folder").reIndex()

Repackaging...


Loading Images...:   0%|          | 0/7000 [00:00<?, ?it/s]

Loading Images...:   0%|          | 0/7000 [00:00<?, ?it/s]

Loading Images...:   0%|          | 0/7000 [00:00<?, ?it/s]

Loading Images...:   0%|          | 0/7000 [00:00<?, ?it/s]

...

Loading Images...:   0%|          | 0/7000 [00:00<?, ?it/s]

Loading Images...:   0%|          | 0/7000 [00:00<?, ?it/s]

Loading Images...:   0%|          | 0/7000 [00:00<?, ?it/s]

Loading Images...:   0%|          | 0/2895 [00:00<?, ?it/s]

Finished Repackaging Bruker Tiffs


Load Images

In [8]:
images = PreProcessing.loadAllTiffs(OutputDirectory)

Loading Images...:   0%|          | 0/9 [00:00<?, ?it/s]

Filter Images

In [9]:
images = PreProcessing.blockwiseFastFilterTiff(images, Footprint=np.ones((7, 3, 3)))

Filtering Images...:   0%|          | 0/3 [00:00<?, ?it/s]

Remove Shutter Artifact

In [10]:
images = PreProcessing.removeShuttleArtifact(images, chunk_size=7000, artifact_length=1000)

Save Images as Binary

In [11]:
PreProcessing.saveRawBinary(images, OutputDirectory)
EH.Encoding.folder_dictionary.get("compiled_imaging_data_folder").reIndex()
EH.recordMod("Repackaged, filtered, and exported images as raw binary. Made video even length this time")

Finished saving video as a binary.


## Image Processing

Imports

In [4]:
from ImagingAnalysis.Suite2PAnalysis import Suite2PModule

Construct Folder for Data

In [4]:
EH.Encoding.addImageSamplingFolder(30)

Motion Correct

In [5]:
MotionCorrection = Suite2PModule(EH.Encoding.folder_dictionary.get("compiled_imaging_data_folder").path, EH.Encoding.Imaging_30Hz.path, file_type="binary")
MotionCorrection.motionCorrect()
MotionCorrection.exportCroppedCorrection(MotionCorrection.ops)
del MotionCorrection # Clean Up

Reference frame, 32.47 sec.
Registered 7000/56000 in 51.38s
Registered 14000/56000 in 103.20s
Registered 21000/56000 in 154.99s
Registered 28000/56000 in 206.90s
Registered 35000/56000 in 261.91s
Registered 42000/56000 in 317.82s
Registered 49000/56000 in 373.88s
Registered 56000/56000 in 429.61s
Exported Cropped Motion-Corrected Video


Denoise

In [3]:
from ImagingAnalysis.Denoising import DenoisingModule

In [4]:
Denoiser = DenoisingModule("ModelForPyTorch", "binary_video",
                    model_path="C:\\ProgramData\\Anaconda3\\envs\\Calcium-Imaging-Analysis-Pipeline\\pth",
                    data_path="".join([EH.Encoding.Imaging_30Hz.path, "\\suite2p\\plane0"]),
                    output_path="".join([EH.Encoding.Imaging_30Hz.path, "\\denoised"]),
                    image_type="binary",
                    length="21000",
                    workers=4,
                    vram=24,
                    batch_size2=4)
Denoiser.runDenoising()
Denoiser = None

ROI Detection

In [ ]:
S2P = Suite2PModule("".join([EH.Encoding.Imaging_30Hz.path, "\\denoised"]), EH.Encoding.Imaging_30Hz.path, file_type="binary")
S2P.roiDetection()

Trace Extraction

In [ ]:
S2P.extractTraces()

ROI Classification

In [ ]:
S2P.classifyROIs()

Finalize (Required spks.npy to use GUI)

In [ ]:
S2P.spikeExtraction()

Save

In [ ]:
# Derp
S2P.ops["meanImg"] = S2P.ops.get("max_proj")
S2P.ops["xrange"]= [0, 496]
S2P.ops["yrange"]=[0, 500]
# Now Save
S2P.save_files()
EH.Encoding.recordMod()
EH.recordMod("S2P Encoding")
EH.saveHierarchy()